In [ ]:
import os
import matplotlib.pyplot as plt
from PIL import Image, ImageOps


In [ ]:
import numpy as np

In [ ]:
path_val = ("data_folder\\images\\images\\train")

In [ ]:
emotions = os.listdir("data_folder\\images\\images\\train")

In [ ]:
emotions

In [ ]:
#We will be building an extra set of images for running all our algorithms in order to train all our models
#This is just to speed up the process

In [ ]:
min_images = 10000

for i in emotions:
    size_folder = len(os.listdir(os.path.join(path_val, i)))
    min_images = min(min_images, size_folder)
    print(i, size_folder)

print(min_images)

In [ ]:
##For just verification
os.listdir(os.path.join(path_val, 'angry'))

In [ ]:
#Let's bulid a 2 dimensional array containing all the names of the images for each of the emotions
images = [[]]*len(emotions)

for i in range(len(emotions)):
    images[i] = os.listdir(os.path.join(path_val, emotions[i]))


In [ ]:
np.array(Image.open(os.path.join(path_val, emotions[0], images[0][1])))

In [ ]:
#Debugging
images[0]

In [ ]:
from tqdm import tqdm

angry = []
disgust = []
fear = []
happy = []
neutral = []
sad = []
surprise = []
for i in tqdm(range(len(images[0]))):
    angry += [np.array(Image.open(os.path.join(path_val, emotions[0], images[0][i])))]

for i in tqdm(range(len(images[1]))):
    disgust += [np.array(Image.open(os.path.join(path_val, emotions[1], images[1][i])))]

for i in tqdm(range(len(images[2]))):
    fear += [np.array(Image.open(os.path.join(path_val, emotions[2], images[2][i])))]

for i in tqdm(range(len(images[3]))):
    happy += [np.array(Image.open(os.path.join(path_val, emotions[3], images[3][i])))]

for i in tqdm(range(len(images[4]))):
    neutral += [np.array(Image.open(os.path.join(path_val, emotions[4], images[4][i])))]

for i in tqdm(range(len(images[5]))):
    sad += [np.array(Image.open(os.path.join(path_val, emotions[5], images[5][i])))]

for i in tqdm(range(len(images[6]))):
    surprise += [np.array(Image.open(os.path.join(path_val, emotions[6], images[6][i])))]

images_array = [angry, disgust, fear, happy, neutral, sad, surprise]

In [ ]:
#Building numpy arrays for training and testing from the orifinal values
from tqdm import notebook

y = []
x = []
for i in notebook.tqdm(range(len(images_array))):
    for j in notebook.tqdm(range(len(images_array[i]))):
        y += [i]
        x += [images_array[i][j]]

In [ ]:
y = np.array(y)
x = np.array(x)
print(x.shape)
print(y.shape)

In [ ]:
from sklearn.model_selection import train_test_split

#For validation set and training set
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=40)

In [ ]:
#One hot encoding our values
import tensorflow as tf
print(np.unique(y_train))
Y_train_onehot = np.array(tf.keras.utils.to_categorical(y_train, num_classes = 7))
Y_val_onehot = np.array(tf.keras.utils.to_categorical(y_val, num_classes = 7))

In [ ]:
np.array(angry).shape

In [ ]:
last_ = os.listdir(os.path.join(path_val, emotions[len(emotions) - 1]))

In [ ]:
np.array(Image.open(os.path.join(path_val, emotions[len(emotions) - 1], last_[len(last_) - 1]))).shape

In [ ]:
compact_training_data = np.array([])


In [ ]:
#Now we have multiple ideas, so working on them in the follwoing cells

In [ ]:
from tensorflow.keras.layers import  Activation, Dense, Flatten, Dropout, BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential, Model

In [ ]:
model1 = Sequential()

model1.add(Conv2D(32, kernel_size = (3, 3), activation = 'relu', input_shape = (48, 48, 1)))
model1.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu'))
model1.add(MaxPooling2D(pool_size = (2, 2)))
model1.add(Dropout(0.25))
model1.add(Conv2D(128, kernel_size = (3, 3), activation = 'relu'))
model1.add(MaxPooling2D(pool_size = (2, 2)))
model1.add(Conv2D(128, kernel_size = (3, 3), activation = 'relu'))
model1.add(MaxPooling2D(pool_size = (2, 2)))
model1.add(Dropout(0.25))
model1.add(Flatten())
model1.add(Dense(1024, activation = 'relu'))
model1.add(Dropout(0.5))
model1.add(Dense(7, activation = 'softmax'))

model1.compile(loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
history = model1.fit(x_train/255., Y_train_onehot, epochs = 8, validation_data = (x_val/255., Y_val_onehot))

In [ ]:
y_pred_val = model1.predict(x_val/255.)

In [ ]:
(y_pred_val[5])

In [ ]:
from PIL import Image as im

print(y_val[0])
print(y_pred_val[0])
im.fromarray(x_val[0])


In [ ]:
max_vals = []
for i in range(len(y_pred_val)):
    max_vals += [np.max(y_pred_val[i])]

max_vals = np.array(max_vals)
#print(max_vals)
print(np.count_nonzero(max_vals < 0.5))
print(len(max_vals))

In [ ]:
not_clear_vals = np.where(max_vals < 0.5)

In [ ]:
unclear_category = []
for i in not_clear_vals:
    unclear_category += [y_val[i]]

unclear_category

In [ ]:
unclear_category[0]

In [ ]:
freq = [0] * 7
for i in unclear_category[0]:
    freq[i] = freq[i] + 1

freq

In [ ]:
np.where(max_vals < 0.5)

In [ ]:
from keras.optimizers import Adam,SGD,RMSprop
from tensorflow import keras

model = Sequential()

#1st CNN layer
model.add(Conv2D(64,(3,3),padding = 'same', activation = 'relu', input_shape = (48,48,1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

#2nd CNN layer
model.add(Conv2D(128,(3,3), activation = 'relu',padding = 'same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout (0.25))

#3rd CNN layer
model.add(Conv2D(128,(3,3), activation = 'relu',padding = 'same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout (0.25))

#4th CNN layer
model.add(Conv2D(256,(3,3), activation = 'relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

#Fully connected 1st layer
model.add(Dense(1024))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))


# Fully connected layer 2nd layer
# model.add(Dense(512))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(Dropout(0.25))

model.add(Dense(no_of_classes, activation='softmax'))



# opt = Adam(lr = 0.0001)
optimizer = keras.optimizers.SGD(learning_rate=0.001, momentum = 0.5)  # replace "lr" with "learning_rate"
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
history2 = model.fit(x_train/255., Y_train_onehot, epochs = 8, validation_data = (x_val/255., Y_val_onehot))